# IBM Data Science: Capstone Project
## The Capstone Notebook


### Introduction

Before COVID-19 threw all other business strategies or plans into complete disarray, Europe was still getting to grips on how Brexit was going to affect the region.

Now that Britain has chosen to leave the EU, many businesses will relocate their head offices outside of London, the world's former financial centre. With this relocation, hundreds of thousands (if not millions) of people will have to uplift their lives and move to another city.

Based on this [forbes article](https://fortune.com/longform/brexit-amsterdam-the-new-london-europe-companies/) Amsterdam could be the 'new London'. If employees are forced to move from the one city to the other, the big question is: which neighborhoods should these soon-to-be ex-Londoners choose to move to in Amsterdam?

Based on Foursquare data, I aim to cluster Amsterdam and London neighborhoods based on similarity. So if you love your neighborhood in London and want something similar in Amsterdam, this is your go-to guide.

### Data

The data I will be using is from accessing the Foursquare API. I will be exploring nighborhoods in London and Amsterdam, using the **explore** function to get the most common venue categories in each neighborhood in the two cities, and then use this feature to group the neighborhoods into clusters. 


I will use the geopy library to get the latitude and longitude values of London and Amsterdam, then use an unsupervised machine learning clustering algorithm, *k*-means, to do the custering. Lastly, I will use the Folium library to colour code the neighborhoods in Amsterdam and London based on their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in London and Amsterdam</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

In [467]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [468]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import GoogleV3 # convert an address into latitude and longitude values

In [469]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [470]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset
### a) Scraping Geo-location Data
### b) Scraping Foursquare Data

First we have to scrape the neighborhood, borough and latitude/longitude data for each city.
* To get the London data I scraped [this](https://en.wikipedia.org/wiki/List_of_areas_of_London) wikipedia page.
* To get the Amsterdam data I scraped [this](https://en.wikipedia.org/wiki/Boroughs_of_Amsterdam) wikipedia page.

Then we have to get the foursquare data

### London

In [471]:
#Scraping the London data
#Reading in the table and setting the header to the first row
df = pd.read_html(r'https://en.wikipedia.org/wiki/List_of_areas_of_London', match = 'Location')
df = pd.DataFrame(df[0])
df.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [472]:
#setting the column labels
df.columns = ['location', 'borough', 'posttown', 'postcode', 'dialcode', 'OSgridref']
df.head()

,location,borough,posttown,postcode,dialcode,OSgridref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [473]:
#dropping unnecessary columns
df.drop(['dialcode', 'OSgridref'], axis =1, inplace = True)
df.columns

Index(['location', 'borough', 'posttown', 'postcode'], dtype='object')

In [474]:
#Only selecting the first postal code and first boroughs listed
first_postcode= df.postcode.str.split(",", expand = True)[0]
df['postcode'] = first_postcode

first_borough= df.borough.str.split(",", expand = True)[0]
df['borough'] = first_borough
df.head()

,location,borough,posttown,postcode
0,Abbey Wood,Bexley,LONDON,SE2
1,Acton,Ealing,LONDON,W3
2,Addington,Croydon[8],CROYDON,CR0
3,Addiscombe,Croydon[8],CROYDON,CR0
4,Albany Park,Bexley,"BEXLEY, SIDCUP",DA5


In [475]:
# Cleaning up the borough names to remove the square brackets with regex
df.borough = df.borough.map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
df

,location,borough,posttown,postcode
0,Abbey Wood,Bexley,LONDON,SE2
1,Acton,Ealing,LONDON,W3
2,Addington,Croydon,CROYDON,CR0
3,Addiscombe,Croydon,CROYDON,CR0
4,Albany Park,Bexley,"BEXLEY, SIDCUP",DA5
5,Aldborough Hatch,Redbridge,ILFORD,IG2
6,Aldgate,City,LONDON,EC3
7,Aldwych,Westminster,LONDON,WC2
8,Alperton,Brent,WEMBLEY,HA0
9,Anerley,Bromley,LONDON,SE20


In [476]:
#only selecting London Neighborhoods and cross referencing that the selection is correct
df = df[df.posttown == "LONDON"]
df.posttown.value_counts()

LONDON    299
Name: posttown, dtype: int64

In [477]:
#dropping the posttown field as it doesnt add value anymore
df.drop(['posttown'], axis =1, inplace = True)
df.reset_index(drop=True, inplace = True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [478]:
df.head()

,location,borough,postcode
0,Abbey Wood,Bexley,SE2
1,Acton,Ealing,W3
2,Aldgate,City,EC3
3,Aldwych,Westminster,WC2
4,Anerley,Bromley,SE20


In [479]:
# The code was removed by Watson Studio for sharing.

In [480]:
from geopy.geocoders import GoogleV3 # convert an address into latitude and longitude values

In [481]:
geolocator = GoogleV3(api_key = googlemaps_api_key, domain = 'maps.google.co.uk')
location = geolocator.geocode('SE2, London, United Kingdom')
location.latitude, location.longitude

(51.4860421, 0.1202586)

In [482]:
def get_lat_long_london(postcode):
    # Initialize the Location (lat. and long.) to "None"
    lat_lng_coords = None
    
    # While loop helps to create a continous run until all the location coordinates are geocoded
    while(lat_lng_coords is None):
        address = '{}, London, United Kingdom'.format(postcode)
        geolocator = GoogleV3(api_key = googlemaps_api_key, domain = 'maps.google.co.uk')
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        lat_lng_coords = [latitude, longitude]
    return lat_lng_coords


In [483]:
#testing
get_lat_long_london('SE2')

[51.4860421, 0.1202586]

In [ ]:
#Using comprehension to create the data for our additional columns
post_codes = [get_lat_long_london(p) for p in df['postcode'].to_list()]

In [ ]:
#creating the dataframe which we will append to the end of our existing dataframe
df_c = pd.DataFrame(post_codes, columns = ['latitude', 'longitude'])
df_c.head()

,latitude,longitude
0,51.486042,0.120259
1,51.512075,-0.267572
2,51.515987,-0.078379
3,51.510583,-0.126768
4,51.412571,-0.061397


In [ ]:
df_lon = pd.concat([df, df_c], axis=1)
df_lon.head()

,location,borough,postcode,latitude,longitude
0,Abbey Wood,Bexley,SE2,51.486042,0.120259
1,Acton,Ealing,W3,51.512075,-0.267572
2,Aldgate,City,EC3,51.515987,-0.078379
3,Aldwych,Westminster,WC2,51.510583,-0.126768
4,Anerley,Bromley,SE20,51.412571,-0.061397


### Amsterdam

In [ ]:
#Scraping the Amsterdam data
#Reading in the table and setting the header to the first row
df = pd.read_html(r'https://en.wikipedia.org/wiki/Boroughs_of_Amsterdam', match = 'Area')
df = pd.DataFrame(df[0])
df.head()

,Borough,Area,Population,Population density,Location (in green),Neighbourhoods
0,Centrum (Centre),8.04 km²,86422,"13,748/km²",NaN,"Binnenstad, Grachtengordel, Haarlemmerbuurt, J..."
1,Noord (North),49.01 km²,94766,"2,269/km²",NaN,"Banne Buiksloot, Buiksloot, Buikslotermeer, Fl..."
2,Nieuw-West(New West),32.38 km²,151677,"4,478/km²",NaN,"Geuzenveld, Nieuw Sloten, Oostoever, Osdorp, O..."
3,Oost (East),30.56 km²,135767,"7,635/km²",NaN,"IJburg, Indische Buurt, Eastern Docklands, Oud..."
4,West,9.89 km²,143842,"15,252/km²",NaN,"Frederik Hendrikbuurt, Houthaven, Spaarndammer..."


In [ ]:
#renaming the columns fo ease of use
df.columns = ['borough', 'area', 'population', 'populationdensity', 'location', 'neighborhoods']
df.head()

,borough,area,population,populationdensity,location,neighborhoods
0,Centrum (Centre),8.04 km²,86422,"13,748/km²",NaN,"Binnenstad, Grachtengordel, Haarlemmerbuurt, J..."
1,Noord (North),49.01 km²,94766,"2,269/km²",NaN,"Banne Buiksloot, Buiksloot, Buikslotermeer, Fl..."
2,Nieuw-West(New West),32.38 km²,151677,"4,478/km²",NaN,"Geuzenveld, Nieuw Sloten, Oostoever, Osdorp, O..."
3,Oost (East),30.56 km²,135767,"7,635/km²",NaN,"IJburg, Indische Buurt, Eastern Docklands, Oud..."
4,West,9.89 km²,143842,"15,252/km²",NaN,"Frederik Hendrikbuurt, Houthaven, Spaarndammer..."


In [ ]:
#dropping columns that I don't need
df = df[['borough','neighborhoods']]
df.head()

,borough,neighborhoods
0,Centrum (Centre),"Binnenstad, Grachtengordel, Haarlemmerbuurt, J..."
1,Noord (North),"Banne Buiksloot, Buiksloot, Buikslotermeer, Fl..."
2,Nieuw-West(New West),"Geuzenveld, Nieuw Sloten, Oostoever, Osdorp, O..."
3,Oost (East),"IJburg, Indische Buurt, Eastern Docklands, Oud..."
4,West,"Frederik Hendrikbuurt, Houthaven, Spaarndammer..."


In [ ]:
#We now need to rearrange the table so that we only have a neighborhood and borough column.
#We start by splitting the neighborhood column into many columns, and add the 'neighborhood' prefix for ease of use
df_split= df.neighborhoods.str.split(",", expand = True)
df_split = df_split.add_prefix('Neighborhood_')
df_split.head()

,Neighborhood_0,Neighborhood_1,Neighborhood_2,Neighborhood_3,Neighborhood_4,Neighborhood_5,Neighborhood_6,Neighborhood_7,Neighborhood_8,Neighborhood_9,Neighborhood_10,Neighborhood_11,Neighborhood_12,Neighborhood_13
0,Binnenstad,Grachtengordel,Haarlemmerbuurt,Jodenbuurt,Jordaan,Kadijken,Lastage,Oosterdokseiland,Oostelijke Eilanden,Plantage,Rapenburg,Uilenburg,Westelijke Eilanden,Weteringschans
1,Banne Buiksloot,Buiksloot,Buikslotermeer,Floradorp,Kadoelen,Molenwijk,Nieuwendam,Nieuwendammerdijk en Buiksloterdijk,Oostzanerwerf,Overhoeks,Tuindorp Nieuwendam,Tuindorp Oostzaan,None,None
2,Geuzenveld,Nieuw Sloten,Oostoever,Osdorp,Overtoomse Veld,Sloten,Slotermeer,Slotervaart,None,None,None,None,None,None
3,IJburg,Indische Buurt,Eastern Docklands,Oud-Oost,Watergraafsmeer,None,None,None,None,None,None,None,None,None
4,Frederik Hendrikbuurt,Houthaven,Spaarndammerbuurt,Staatsliedenbuurt,Zeeheldenbuurt,Westerpark,Kinkerbuurt,Overtoombuurt,De Baarsjes,Bos en Lommer,Kolenkitbuurt,Landlust,Sloterdijk,None


In [ ]:
#we then join the dataframes back together again and drop the repetitive 'neighborhoods' columns
df_new = pd.concat([df, df_split], axis = 1)
df_new.drop(['neighborhoods'], axis = 1, inplace = True)
df_new

,borough,Neighborhood_0,Neighborhood_1,Neighborhood_2,Neighborhood_3,Neighborhood_4,Neighborhood_5,Neighborhood_6,Neighborhood_7,Neighborhood_8,Neighborhood_9,Neighborhood_10,Neighborhood_11,Neighborhood_12,Neighborhood_13
0,Centrum (Centre),Binnenstad,Grachtengordel,Haarlemmerbuurt,Jodenbuurt,Jordaan,Kadijken,Lastage,Oosterdokseiland,Oostelijke Eilanden,Plantage,Rapenburg,Uilenburg,Westelijke Eilanden,Weteringschans
1,Noord (North),Banne Buiksloot,Buiksloot,Buikslotermeer,Floradorp,Kadoelen,Molenwijk,Nieuwendam,Nieuwendammerdijk en Buiksloterdijk,Oostzanerwerf,Overhoeks,Tuindorp Nieuwendam,Tuindorp Oostzaan,None,None
2,Nieuw-West(New West),Geuzenveld,Nieuw Sloten,Oostoever,Osdorp,Overtoomse Veld,Sloten,Slotermeer,Slotervaart,None,None,None,None,None,None
3,Oost (East),IJburg,Indische Buurt,Eastern Docklands,Oud-Oost,Watergraafsmeer,None,None,None,None,None,None,None,None,None
4,West,Frederik Hendrikbuurt,Houthaven,Spaarndammerbuurt,Staatsliedenbuurt,Zeeheldenbuurt,Westerpark,Kinkerbuurt,Overtoombuurt,De Baarsjes,Bos en Lommer,Kolenkitbuurt,Landlust,Sloterdijk,None
5,Westpoort(West Gateway),Westpoort,None,None,None,None,None,None,None,None,None,None,None,None,None
6,Zuid (South),Apollobuurt,Buitenveldert,Hoofddorppleinbuurt,Museumkwartier,De Pijp,Prinses Irenebuurt,Rivierenbuurt,Schinkelbuurt,Stadionbuurt,Vondelpark,Willemspark,Zuidas,None,None
7,Zuidoost(Southeast),Bijlmermeer,Venserpolder,Gaasperdam,Driemond,None,None,None,None,None,None,None,None,None,None


In [ ]:
#we then use melt to get our two columns
df_new.set_index('borough', inplace =  True)
df_new.head()
#df_new.pivot(index = 'borough', columns = df.columns[1:])

,Neighborhood_0,Neighborhood_1,Neighborhood_2,Neighborhood_3,Neighborhood_4,Neighborhood_5,Neighborhood_6,Neighborhood_7,Neighborhood_8,Neighborhood_9,Neighborhood_10,Neighborhood_11,Neighborhood_12,Neighborhood_13
borough,,,,,,,,,,,,,,
Centrum (Centre),Binnenstad,Grachtengordel,Haarlemmerbuurt,Jodenbuurt,Jordaan,Kadijken,Lastage,Oosterdokseiland,Oostelijke Eilanden,Plantage,Rapenburg,Uilenburg,Westelijke Eilanden,Weteringschans
Noord (North),Banne Buiksloot,Buiksloot,Buikslotermeer,Floradorp,Kadoelen,Molenwijk,Nieuwendam,Nieuwendammerdijk en Buiksloterdijk,Oostzanerwerf,Overhoeks,Tuindorp Nieuwendam,Tuindorp Oostzaan,None,None
Nieuw-West(New West),Geuzenveld,Nieuw Sloten,Oostoever,Osdorp,Overtoomse Veld,Sloten,Slotermeer,Slotervaart,None,None,None,None,None,None
Oost (East),IJburg,Indische Buurt,Eastern Docklands,Oud-Oost,Watergraafsmeer,None,None,None,None,None,None,None,None,None
West,Frederik Hendrikbuurt,Houthaven,Spaarndammerbuurt,Staatsliedenbuurt,Zeeheldenbuurt,Westerpark,Kinkerbuurt,Overtoombuurt,De Baarsjes,Bos en Lommer,Kolenkitbuurt,Landlust,Sloterdijk,None


In [ ]:
#Cleaning up the dataframe
df = pd.DataFrame(df_new.stack(level=0))
df.index = df.index.get_level_values('borough')
df.reset_index(drop =False, inplace =True)
df.rename(columns = {0:'neighborhood'}, inplace = True)
df.head()

,borough,neighborhood
0,Centrum (Centre),Binnenstad
1,Centrum (Centre),Grachtengordel
2,Centrum (Centre),Haarlemmerbuurt
3,Centrum (Centre),Jodenbuurt
4,Centrum (Centre),Jordaan


In [ ]:
#we now need to remove the english translation within the brackets by using regex
df['borough'] = df['borough'].str.replace(r'\([^()]*\)', '')
df.head()

,borough,neighborhood
0,Centrum,Binnenstad
1,Centrum,Grachtengordel
2,Centrum,Haarlemmerbuurt
3,Centrum,Jodenbuurt
4,Centrum,Jordaan


In [ ]:
#Now using the get_lat_lng function we used earlier on the amsterdam data, we have to change the domain and address string:
geolocator = GoogleV3(api_key = googlemaps_api_key, domain = 'maps.google.co.uk')

In [ ]:
def get_lat_long_amsterdam(neighborhood):
    # Initialize the Location (lat. and long.) to "None"
    lat_lng_coords = None
    
    # While loop helps to create a continous run until all the location coordinates are geocoded
    while(lat_lng_coords is None):
        address = '{}, Amsterdam, Nederland'.format(neighborhood)
        geolocator = GoogleV3(api_key = googlemaps_api_key, domain = 'maps.google.co.uk')
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        lat_lng_coords = [latitude, longitude]
    return lat_lng_coords


In [ ]:
#checking that it works
location = geolocator.geocode('Grachtengordel, Amsterdam, Nederland')
location.latitude, location.longitude
print (location.latitude, location.longitude)
print('It works!')

52.3655774 4.8893609
It works!


In [ ]:
#Using comprehension to create the data for our additional columns
post_codes = [get_lat_long_amsterdam(n) for n in df['neighborhood'].to_list()]
post_codes.head()

In [ ]:
#creating the dataframe which we will append to the end of our existing dataframe
df_am_lat_lng = pd.DataFrame(post_codes, columns = ['latitude', 'longitude'])
df_am_lat_lng.head()

In [ ]:
df_am = pd.concat([df, df_am_lat_lng], axis=1)
df_am.head()

### Now concatenating the two dataframes by row (on top of one another), we get the final dataframe

In [ ]:
df_lon.head()

In [ ]:
df_am.head()

In [ ]:
df_am = df_am[['neighborhood', 'borough', 'latitude', 'longitude']]
df_am['city'] = 'Amsterdam'
df_am.head()

In [ ]:
#Because we dont have the post codes for Amsterdam- and given that we actually don't need them , we can drop it before we concatenate.
df_lon.drop(['postcode'], axis = 1, inplace =True)

In [ ]:
df_lon.rename({'location': 'neighborhood'}, axis = 1, inplace = True)
df_lon['city'] = 'london'

In [ ]:
df_am_lon = pd.concat([df_lon, df_am])
df_am_lon.reset_index(drop =True, inplace =True)
df_am_lon

In [ ]:
# Having a look at the two cities using a folium map
#Looking at Amsterdam first
location = geolocator.geocode( 'Amsterdam, Nederland')
lat_am = location.latitude
lng_am = location.longitude
print (lat_am, lng_am)

In [ ]:
# create map of Amsterdam using latitude and longitude values
map_am= folium.Map(location=[lat_am, lng_am], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_am['latitude'], df_am['longitude'], df_am['borough'], df_am['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_am)  
    
map_am

In [ ]:
# Having a look at the two cities using a folium map
#Looking at London
location = geolocator.geocode( 'London, United Kingdom')
lat_lon = location.latitude
lng_lon = location.longitude
print (lat_lon, lng_lon)

In [ ]:
# create map of Amsterdam using latitude and longitude values
map_lon= folium.Map(location=[lat_lon, lng_lon], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_lon['latitude'], df_lon['longitude'], df_lon['borough'], df_lon['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lon)  
    
map_lon

In [ ]:
df_am_lon.shape

In [ ]:
df_am_lon.borough.nunique()

In [ ]:
df_am_lon.groupby(by = ['city']).count()['neighborhood']

In [ ]:
a = df_am_lon.groupby(by = ['city'])['borough'].value_counts()

### Findings from our initial neighborhood data

We now have a final dataframe with  368 neighborhoods across both cities.
Given the size of London compared to Amsterdam, it's no surprise that London has over 3 times as many neighborhoods.
* London has 299 Neighborhoods
* Amsterdam has 69 Neighborhoods

In total, there are 41 unique Boroughs between the two cities.
* The Borough with the most neighborhoods in Amsterdam is Centrum with 14 Neighborhoods.
* The Borough with the most neighborhoods in London is Barnet with 26 Neighborhoods.

If we take a look at each folium map, we can see that London as a geographic region is much bigger than Amsterdam.

## 3. Clustering Neighborhoods using FourSquare <a class="anchor" id="third-bullet"></a>

In [ ]:
import requests # library to handle requests
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
#Exploring the first neighborhood in our dataset- 'Abbey Wood'

df_am_lon.loc[0, 'neighborhood']

#### Now, let's get the top 50 venues that are in Regent Park within a radius of 500 meters.

In [ ]:
neighborhood_name = df_am_lon.loc[0, 'neighborhood']
neighborhood_latitude = df_am_lon.loc[0, 'latitude']
neighborhood_longitude = df_am_lon.loc[0, 'longitude']

LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

In [ ]:
#Sending the GET request to examine the results
results = requests.get(url).json()
results

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
#Cleaning Json and putting into pandas DF structure
venues = results['response']['groups'][0]['items']
venues

In [ ]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues.head()

In [ ]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

In [ ]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.head()

In [ ]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.columns

In [ ]:
nearby_venues.head()

In [ ]:
#Creating a function to repeat the same for all neighborhoods in our London and Amsterdam dataset
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
am_lon_venues = getNearbyVenues(names=df_am_lon['neighborhood'],
                                   latitudes=df_am_lon['latitude'],
                                   longitudes=df_am_lon['longitude']
                                  )


In [ ]:
#Sizing and getting a view of the data
print(am_lon_venues.shape)
am_lon_venues.head()

In [ ]:
#Seeing how many venues we get per neighborhood
am_lon_venues.groupby('Neighborhood').count().head()

#### Analysing each neighborhood

In [ ]:
# one hot encoding
am_lon_onehot = pd.get_dummies(am_lon_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
am_lon_onehot['Neighborhood'] = am_lon_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [am_lon_onehot.columns[-1]] + list(am_lon_onehot.columns[:-1])
am_lon_onehot = am_lon_onehot[fixed_columns]

am_lon_onehot.head()

In [ ]:
#Grouping the rows by neighborhood and taking the mean of the frequency of the occurance
am_lon_grouped = am_lon_onehot.groupby('Neighborhood').mean().reset_index()
am_lon_grouped.head()

In [ ]:
#Printing each neighborhood along with the 5 most common venues
num_top_venues = 5

for hood in am_lon_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = am_lon_grouped[am_lon_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
#Writing a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
#Creating a new DF to display the top venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = am_lon_grouped['Neighborhood']

for ind in np.arange(am_lon_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(am_lon_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

#### Now that our data is in a workable format, we'll start our clustering

In [ ]:
# set number of clusters
kclusters = 5

am_lon_grouped_clustering = am_lon_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(am_lon_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
#creating anew dataframe that includes cluster and the top 10 venues per neighborhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
am_lon_merged.rename(columns = {'neighborhood':'Neighborhood'}, inplace = True)

In [ ]:
neighborhoods_venues_sorted.head()

In [ ]:
am_lon_merged = df_am_lon

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
am_lon_merged = am_lon_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

am_lon_merged # check the last columns!

In [ ]:
am_lon_merged.dropna(inplace = True)

In [ ]:
am_lon_merged['Cluster Labels'] = am_lon_merged['Cluster Labels'].astype(int)

In [ ]:
# create map for Amsterdam
map_clusters_am = folium.Map(location=[lat_am, lng_am], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(am_lon_merged['latitude'], am_lon_merged['longitude'], am_lon_merged['Neighborhood'], am_lon_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters_am

In [ ]:
# create map for London
map_clusters_lon = folium.Map(location=[lat_lon, lng_lon], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(am_lon_merged['latitude'], am_lon_merged['longitude'], am_lon_merged['Neighborhood'], am_lon_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters_lon

#### Examining Clusters

In [ ]:
#Cluster 1
am_lon_merged.loc[am_lon_merged['Cluster Labels'] == 0, am_lon_merged.columns[[1] + list(range(5, am_lon_merged.shape[1]))]]

In [ ]:
#Cluster 2
am_lon_merged.loc[am_lon_merged['Cluster Labels'] == 1, am_lon_merged.columns[[1] + list(range(5, am_lon_merged.shape[1]))]]

In [ ]:
#Cluster 3
am_lon_merged.loc[am_lon_merged['Cluster Labels'] == 2, am_lon_merged.columns[[1] + list(range(5, am_lon_merged.shape[1]))]]

In [ ]:
#Cluster 4
am_lon_merged.loc[am_lon_merged['Cluster Labels'] == 3, am_lon_merged.columns[[1] + list(range(5, am_lon_merged.shape[1]))]]

In [ ]:
#Cluster 5
am_lon_merged.loc[am_lon_merged['Cluster Labels'] == 4, am_lon_merged.columns[[1] + list(range(5, am_lon_merged.shape[1]))]]

### Looking at the clusters, we have many the most homogeneity in cluster 1 meaning that those boroughs are probably the most similar. If you are looking for something a bit different, it seems that Boroughs in cluster 2-5 would be something to look into.